# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846865144028                   -0.70    4.8
  2   -7.852323738575       -2.26       -1.53    1.0
  3   -7.852646201525       -3.49       -2.52    3.0
  4   -7.852646677740       -6.32       -3.37    2.5
  5   -7.852646685801       -8.09       -4.73    1.0
  6   -7.852646686723       -9.04       -5.20    3.5
  7   -7.852646686730      -11.16       -6.17    1.5
  8   -7.852646686730      -12.60       -7.48    2.5
  9   -7.852646686730      -14.27       -7.58    2.0
 10   -7.852646686730   +  -14.57       -8.55    2.0
 11   -7.852646686730      -14.57       -9.53    2.2
 12   -7.852646686730   +  -14.75      -10.08    3.0
 13   -7.852646686730   +  -15.05      -10.97    1.2
 14   -7.852646686730      -15.05      -11.62    2.2
 15   -7.852646686730      -14.75      -12.82    3.0


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846808419841                   -0.70           4.8
  2   -7.852525474992       -2.24       -1.63   0.80    2.0
  3   -7.852635679345       -3.96       -2.71   0.80    1.0
  4   -7.852646373827       -4.97       -3.28   0.80    2.2
  5   -7.852646680535       -6.51       -4.07   0.80    1.5
  6   -7.852646686226       -8.24       -4.72   0.80    1.2
  7   -7.852646686716       -9.31       -5.68   0.80    1.8
  8   -7.852646686730      -10.88       -6.70   0.80    2.2
  9   -7.852646686730      -12.57       -7.16   0.80    2.5
 10   -7.852646686730      -14.35       -7.95   0.80    1.5
 11   -7.852646686730   +    -Inf       -8.16   0.80    1.8
 12   -7.852646686730      -14.45       -8.98   0.80    1.2
 13   -7.852646686730   +  -14.75       -9.43   0.80    1.8
 14   -7.852646686730   +    -Inf      -10.77   0.80    1.5
 15   -7.852646686730   +    -Inf      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.378218e+01     3.244297e+00
 * time: 0.5062358379364014
     1     1.554891e+00     1.710487e+00
 * time: 0.7610268592834473
     2    -1.301445e+00     2.099653e+00
 * time: 0.7886350154876709
     3    -3.476257e+00     1.857086e+00
 * time: 0.8282878398895264
     4    -4.603163e+00     1.839292e+00
 * time: 0.8675458431243896
     5    -6.400771e+00     1.335774e+00
 * time: 0.9073469638824463
     6    -7.178666e+00     8.079101e-01
 * time: 0.9476280212402344
     7    -7.377928e+00     9.027484e-01
 * time: 0.9742538928985596
     8    -7.559123e+00     5.509254e-01
 * time: 1.0001449584960938
     9    -7.606898e+00     5.409599e-01
 * time: 1.026322841644287
    10    -7.698312e+00     1.824249e-01
 * time: 1.0523350238800049
    11    -7.722556e+00     7.170414e-02
 * time: 1.0784709453582764
    12    -7.740820e+00     9.024789e-02
 * time: 1.1047029495239258
    13    -7.771593e+00     1.805883e-01
 * time: 1.13077187538

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846715392565                   -0.70    4.5
  2   -7.852304134981       -2.25       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686702                   -2.55
  2   -7.852646686730      -10.56       -5.90
  3   -7.852646686730   +  -14.75      -12.59


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 4.707649719393637e-14
|ρ_newton - ρ_scfv| = 1.3413029869560504e-13
|ρ_newton - ρ_dm|   = 2.539333807558978e-10
